# Example for PET projection and back-projection using STIR.

This example computes projection data and the back-projection of that
data using the Shepp-Logan phantom in ODL as input. Definition
of the acquisition geometry and computations are done entirely in STIR,
where the communication between ODL and STIR is realized with files
via hard disk.


## From Interfile headers

We assume that there are both volume and sinogram header files available.
Note that we _don't_ need the corresponding data files.

In [ ]:
from pathlib import Path

In [ ]:
data_path = Path() / 'data' / 'stir'

In [ ]:
projection_path = data_path / 'small.hs'

In [ ]:
volume_path = data_path / 'initial.hv'

In [ ]:
from odlpet.stir.io import projector_from_file

In [ ]:
proj = projector_from_file(volume_path.as_posix(), projection_path.as_posix())

In [ ]:
proj.domain

In [ ]:
proj.range

In [ ]:
from odlpet.utils.phantom import derenzo

In [ ]:
vol = derenzo(proj.domain)

In [ ]:
%matplotlib inline

In [ ]:
vol.show(coords=(.5,None,None), aspect='equal')

In [ ]:
data = proj(vol)

In [ ]:
data.show(indices=(0,None,None))

## Back projection

In [ ]:
proj.adjoint(data).show(coords=(3,None,None), aspect='equal')

## Using a Compression object

We can also create a compression object. To this end, we have to guess what kind of compression was used to create the data. This may be accessible in the interfile sinogram header though.

In [ ]:
from odlpet.stir.io import _proj_data_info_from_file

In [ ]:
from odlpet.scanner.compression import Compression

In [ ]:
proj_data_info = _proj_data_info_from_file(projection_path.as_posix())

In [ ]:
comp = Compression.from_stir_proj_data_info(proj_data_info)

We still have to guess the right values here:

In [ ]:
comp.num_of_views = 28

comp.max_num_segments = 4

comp.span_num = 3

In [ ]:
proj_ = comp.get_projector()

In [ ]:
proj__ = comp.get_projector(stir_proj_data_info=proj_data_info)

In [ ]:
proj_.domain.shape == proj__.domain.shape

In [ ]:
proj.range.shape == proj_.range.shape == proj__.range.shape

We can now plot the sinogram for the second view in the first segment:

In [ ]:
data.show(indices=(comp.get_offset(1,2), None, None))

We can also force the domain to be the one specified in the volume header file:

In [ ]:
from odlpet.stir.io import stir_domain_from_file

In [ ]:
proj____ = comp.get_projector(stir_domain=stir_domain_from_file(volume_path.as_posix()), stir_proj_data_info=proj_data_info)

In [ ]:
proj.domain.shape == proj____.domain.shape